<a href="https://colab.research.google.com/github/Xinchunran/GLM/blob/main/TradingViewCrawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Readme
1. This is Colab code, if using other notebook, it may need install more lib.
1. Setup save path at `GOOGLE_DRIVE_SAVE_PATH` in the second code block.
1. Run `crawl_tradingview_by_page(page:int)` with page number, every page have 18 strategies

In [1]:
!pip install fake-useragent
!pip install akshare --upgrade
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 21.2 MB/s eta 0:00:00
  Created wheel for jsonpath: filename=jsonpath-0.82.2-py3-none-any.whl size=5615 sha256=48180dc73f5e77ed948a900a63d19b6f737f6962c41020cbfb1b12f32f444ab6
  Stored in directory: /root/.cache/pip/wheels/a2/e3/6a/59f241672f3221253b1236359b0a711dabb69795a006184398
Successfully built jsonpath
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 h

In [2]:
from google.colab import drive
drive.mount('/content/drive')
# Please set your goolgle path or local path to save the training data
GOOGLE_DRIVE_SAVE_PATH='/content/drive/MyDrive/SolidGPT/TradingViewDataset/'

Mounted at /content/drive


In [3]:
from fake_useragent import UserAgent
import requests
from bs4 import BeautifulSoup
import json
import akshare as ak
import pandas as pd
from random import randint
from time import sleep
from tqdm.notebook import tqdm
import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import logging
import time

In [4]:
from typing_extensions import Text
ua = UserAgent()

def text_format(value : str):
    return ''.join(value.splitlines())

def get_script_url_list_from_page(page: int):
  URL = f'https://www.tradingview.com/scripts/page-{page}/?script_type=strategies'  # replace with your target URL
  headers = {'User-Agent': ua.random}
  response = requests.get(URL, headers=headers)
  soup = BeautifulSoup(response.content, 'html.parser')
  # Find all divs with the specified class
  divs = soup.find_all('p', class_='tv-widget-idea__description-row tv-widget-idea__description-row--clamped js-widget-idea__popup')
  # Extract data-href attribute for each div
  return [div.get('data-href') for div in divs]

def get_strategy_description_from(url: str):
  tradingview_route = 'https://www.tradingview.com'
  URL = f'{tradingview_route}{url}'  # replace with your target URL
  headers = {'User-Agent': ua.random}
  response = requests.get(URL, headers=headers)
  soup = BeautifulSoup(response.content, 'html.parser')
  # Find all divs with the specified class
  divs = soup.find_all('div', class_='tv-chart-view__description selectable')
  plain_texts = [div.get_text() for div in divs]
  res = ""
  for text in plain_texts:
    res += text_format(text)
  return res

def get_strategies_script_from(url: str):
  tradingview_route = 'https://www.tradingview.com'
  url = f'{tradingview_route}{url}'  # replace with your target URL
  # Set up the driver (assumes chromedriver is in your PATH)
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome(options=chrome_options)
  driver.get(url)  # Replace with your target URL
  button = driver.find_element(By.XPATH, '//button[@class="collapseBtn-PWda3OiR button-D4RPB3ZC size-medium-D4RPB3ZC color-gray-D4RPB3ZC variant-primary-D4RPB3ZC stretch-D4RPB3ZC with-start-icon-D4RPB3ZC apply-overflow-tooltip apply-overflow-tooltip--check-children-recursively apply-overflow-tooltip--allow-text"]')
  button.click()
  time.sleep(2)
  # Click the "Click to Copy" button. This assumes the button can be located by its text. Adjust as necessary.
  divs = driver.find_elements(By.XPATH, '//div[@class="view-lines monaco-mouse-cursor-text"]')
  res = ""
  for div in divs:
    res += div.text

  driver.quit()
  return res

In [5]:
get_strategy_description_from('/4TKWH069-Williams-R-Strategy/')


''

In [6]:
def crawl_tradingview_by_page(page:int, path:str = GOOGLE_DRIVE_SAVE_PATH):
  url_list = get_script_url_list_from_page(page)
  print(url_list)
  for url in tqdm(url_list):
    try:
      file_name = url.split('/')[2]
      with open(f'{GOOGLE_DRIVE_SAVE_PATH}Strategy-{file_name}.txt', 'w') as file:
          file.write("Prompt:\n")
          file.write(get_strategy_description_from(url))
          file.write("Completion:\n")
          file.write(get_strategies_script_from(url))
      # random sleep to prevent crawler detector
      time.sleep(randint(1, 3))
    except Exception as e:
      logging.warning(f"Error happen when fetch the strategies script in url: {url} with the error {e}")

In [11]:
# Please change the page num and run
crawl_tradingview_by_page(17)

['/script/SvcAJM3O-Strategy-Myth-Busting-10-InsideBar-EMA-MYN/', '/script/tz9DxyAw-XRP-1h-Chanu-Delta-inspired-Breakeven-Strategy/', '/script/r4SVQcwp/', '/script/nSujiUDb/', '/script/7Ijtq4XE-Average-Directional-Index-v2/', '/script/C0j14hDI-Quadriple-EMA-Strategy/', '/script/2vgZW5T0-Crypto-BTC-Correlation-Scalper-Gaps-Strategy/', '/script/b4qtNKsi-LarryWillians-Simple-Strategy/', '/script/gH3jt4wN-Strategy-Myth-Busting-8-TrendSurfers-TrendOsc-MYN/', '/script/O4vhK1BK/', '/script/TGjaRTeT/', '/script/BNMeQJfS/', '/script/i4msrNrT-Suchit-RSI-CCI/', '/script/eYsUViQA-Morning-Scalp-Strategy/', '/script/qgtXBmhG/', '/script/ekQPU4LF-Fake-Strategy/', '/script/3y2AAyAB-Open-High-Low-Strategy/', '/script/n45eXKBG-Moving-Average-Displaced-Envelope-ATRTS/']


  0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:
# Retry specific strategy
# url = '/script/bhlo7KAz-Last-Price-minus-Open-Price-Intraday-Volume/'
# file_name = url.split('/')[2]
# with open(f'{GOOGLE_DRIVE_SAVE_PATH}Strategy-{file_name}.txt', 'w') as file:
#     file.write("Prompt:\n")
#     file.write(get_strategy_description_from(url))
#     file.write("Completion:\n")
#     file.write(get_strategies_script_from(url))



In [ ]:
for i in range(1, 16):
  print(f"start page: {i}")
  crawl_tradingview_by_page(i)
  print(f"crawl page: {i} done")

start page: 1
['/script/wP7WWjLL-SuperTrend-AI-Clustering-LuxAlgo/', '/script/yRHQ9ufY-Support-Resistance-Dynamic-LuxAlgo/', '/script/J9LyN92e-Master-Pattern-LuxAlgo/', '/script/I2X9DE84-Machine-Learning-Momentum-Index-MLMI-Zeiierman/', '/script/iOrhpIqc-Support-and-Resistance-Signals-MTF-LuxAlgo/', '/script/Bcufg8oX-HyperTrend-LuxAlgo/', '/script/eTgP2ymK-Volume-SuperTrend-AI-Expo/', '/script/aRxQ1g82-QQE-Weighted-Oscillator-LuxAlgo/', '/script/30aLnPNh-AI-Trend-Navigator-K-Neighbor/', '/script/arQP5Q2f-Inverse-FVG-with-Rejections-TFO/', '/script/3ONrWovL-Liquidity-Levels-Voids-VP-LuxAlgo/', '/script/y2WjAnHA-AI-SuperTrend-Clustering-Oscillator-LuxAlgo/', '/script/OhLE0vnH-Volume-Delta-Methods-Chart-LuxAlgo/', '/script/kBHEpriQ-Adaptive-MACD-LuxAlgo/', '/script/58YCiPaa-Machine-Learning-Regression-Trend-LuxAlgo/', '/script/3pQjf5YJ-Fibo-Levels-with-Volume-Profile-and-Targets-ChartPrime/', '/script/5Q0rRNo6-DBMA-Dual-Bollinger-Moving-Average/', '/script/V30Sbp5O-Liquidity-Sentiment-Pro

  0%|          | 0/18 [00:00<?, ?it/s]

crawl page: 1 done
start page: 2
['/script/uWMQ0vwN-Risk-to-Reward-FIXED-SL-Backtester/', '/script/ZSaEDKYt/', '/script/sMutGrrG-tradinghook-Renko-Trend-Reversal-Strategy-V2/', '/script/mA6NEHZE-Buying-Selling-Volume-Strategy/', '/script/mg1Qa51D-Previous-Day-High-Low-Strategy-only-for-Long/', '/script/I4FhUvyq-Quantitative-Trend-Strategy-Uptrend-long/', '/script/DTIiJ6cx-Financial-Ratios-Fundamental-Strategy/', '/script/PJb1qh7h-Strategy-Reversal-Catcher/', '/script/Ce2jjXVU-Merovinh-Mean-Reversion-Lowest-low/', '/script/pCxvfqFv-3-Signal-Directional-Trend-Strategy-for-E-Minis/', '/script/7QZGe04Y-Crunchster-s-Turtle-and-Trend-System/', '/script/B74emdHa-TRAX-Detrended-Price-Strategy/', '/script/TIpnJpU8-PresentTrend-Strategy-presentTrading/', '/script/ttzK15qh-Striker-Play/', '/script/kKxnTEuC-CC-Trend-strategy-2-Downtrend-Short/', '/script/QHGAgBT2-Linear-On-MACD/', '/script/V3cRaSNH-Vortex-Cross-w-MA-Confirmation/', '/script/M6Pz1zvt-Ahsan-Tufail-Precise-MA-Crossover-Filter-for-Rel

  0%|          | 0/18 [00:00<?, ?it/s]

crawl page: 2 done
start page: 3
['/script/fEwucu6l-Trend-Confirmation-Strategy/', '/script/qMumNo8G-Trend-Following/', '/script/owieyhFU-Linear-Cross-Trading-Strategy/', '/script/Xr2xrNAY-Volatility-Capture-RSI-Bollinger-Strategy-presentTrading/', '/script/KwYFiCcZ-Crunchster-s-Normalised-Trend-Strategy/', '/script/sUMz51py/', '/script/ZeaaEnKo-Strategy-Template/', '/script/zFKRj4Gi-Dual-Supertrend-with-MACD-Strategy-presentTrading/', '/script/Tyxno1gE-80-20/', '/script/vOPO3zto-twisted-SMA-strategy-4h/', '/script/ADnS28kf/', '/script/2au7bXud-Buy-Only-Strategy-with-Dynamic-Re-Entry-and-Exit/', '/script/gMVECpa5/', '/script/aksPa3e7-Bollinger-Bands-Modified-Stormer/', '/script/mNsk0Y8K/', '/script/lro9g4Kn-HEFF4MAYOR-TRADING-BOT/', '/script/AI6DyEXY-Moving-Average-Rainbow-Stormer/', '/script/qVnc6vMT-RIBBON-EMA-CROSS-BY-HB/']


...

  0%|          | 0/18 [00:00<?, ?it/s]

crawl page: 4 done
start page: 5
['/script/xha3uEoj-Mechanical-Trading-Strategy/', '/script/cnILutZ3-RSI-CCI-Fusion-Strategy/', '/script/WTmh8KER-Ultimate-Balance-Strategy/', '/script/qQcOhymF-CITI-Moku-Strategy-Free-by-MiaIsTheNewBird/', '/script/17Hd0yWl/', '/script/4TKWH069-Williams-R-Strategy/', '/script/5MQm4L0J-9-22-5-MIN-15-MIN-BANKNIFTY/', '/script/ecRye6YY-HK-Percentile-Interpolation-One/', '/script/FX9JugDa-Moving-Average-Crossover-Strategy/', '/script/hUK5mTUI-BB-and-KC-Strategy/', '/script/QLDOWdxE-Equity-Curve-Trading-with-EMA/', '/script/YexkkFy8-Master-Supertrend-Strategy-Trendoscope/', '/script/aHwncfep-Monthly-Strategy-Performance-Table/', '/script/1rpeHDCL/', '/script/PmJa1OXL/', '/script/DDWqs5FT/', '/script/XoR8rQaW-VWAP-Trendfollow-Strategy-wbburgin/', '/script/9WNF3S0S-Range-Breaker/']


  0%|          | 0/18 [00:00<?, ?it/s]

crawl page: 5 done
start page: 6
['/script/vQSBf9rh-Wyckoff-Range-Strategy/', '/script/E3Z1VhMa-Initial-Balance-Panel-Strategy-for-Bitcoin/', '/script/fgwTwNJM-Volatility-Breakout-Strategy-Angel-Algo/', '/script/TsR0HZC3/', '/script/cCEupOMt-Price-Action-Support-Resistance-MACD-LONG-Strategy/', '/script/GMVL1W8C-Bollinger-Bands-RSI-and-MA-Strategy/', '/script/HPnnfjKx-Advanced-Trend-Detection-Strategy/', '/script/ZJseWDie-Grid-Spot-Trading-Algorithm-V2-The-Quant-Science/', '/script/kkLgubYm-MACD-TrueLevel-Strategy/', '/script/47AmMd56-RSI-TrueLevel-Strategy/', '/script/QJRHUWcF-Chandelier-Exit-ZLSMA-Strategy/', '/script/A1wYJl6k-Strategy-Creator/', '/script/bdrABiYK-Break-even-stop-loss-of-instrument-price/', '/script/nhcUrl6F-SuperTrend-Long-Strategy-TrendFilter/', '/script/6l9kcw7j/', '/script/kQ92NEto-X48-Strategy-ADAPTIVE-CONSECUTIVE-TP-SL-V-1/', '/script/eloUVDFB-Gold-Fair-Value-Gap-Entry-FVG-GOLD/', '/script/sTFw2fOj/']


  0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:
# Generate open ai training set
import json
import os

def create_trainset_file(filename, role_assumption):
    # Define the data structure
    data = {
        "messages": [
            { "role": "system", "content": role_assumption }
        ]
    }
    # Write to the JSON file
    with open(filename, 'w') as file:
        json.dump(data, file, indent=4)

def add_messages_to_json(filename, user_msg, assistant_msg):
    # Load the current content of the JSON file
    with open(filename, 'r') as file:
        data = json.load(file)

    # Append the user and assistant messages
    data["messages"].append({"role": "user", "content": user_msg})
    data["messages"].append({"role": "assistant", "content": assistant_msg})

    # Write the updated content back to the JSON file
    with open(filename, 'w') as file:
        json.dump(data, file, indent=4)

def populate_data_to_gpt_trainset(directory, trainset_path):

    # List all files in the directory
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)

        # Check if it's a file (and not a directory)
        if os.path.isfile(filepath):
            with open(filepath, 'r') as file:
                file_content = file.read()
                prompt, completion = extract_contents(file_content)
                add_messages_to_json(trainset_path, prompt, completion)

def extract_contents(s):
    # Extract content between "Prompt:" and "Completion:"
    prompt_content = re.search(r'Prompt:(.*?)Completion:', s, re.DOTALL)
    # If a match is found, extract it; otherwise, set to None
    prompt_content = prompt_content.group(1).strip() if prompt_content else None
    # Extract content after "Completion:"
    completion_content = re.search(r'Completion:(.*)', s, re.DOTALL)
    # If a match is found, extract it; otherwise, set to None
    completion_content = completion_content.group(1).strip() if completion_content else None
    return prompt_content, completion_content

trainset_path = f'{GOOGLE_DRIVE_SAVE_PATH}GPTTrain/trainset'
create_trainset_file(trainset_path, 'Assume you are Quant Developer and the expert of PineScript v5')
populate_data_to_gpt_trainset(GOOGLE_DRIVE_SAVE_PATH, trainset_path)
